In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 1st link

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

content = requests.get('https://geology.fandom.com/wiki/List_of_rock_types', headers=headers).text
bsoup3 = BeautifulSoup(content, 'html.parser')

In [3]:
def check_rock_exists(rockname, rocks):
    if rockname in rocks:
        return True
    else:
        return False
    
    
def get_rock_name(item, rocks, parentList, parent):
    if(len(list(item.children)) > 1):
        rock_name = list(item.children)[0].text
    else:
        if(item.find("span")):
            rock_name = item.findChildren("span")[0].text
        else:
            rock_name = list(item.children)[0]
    
    if not check_rock_exists(rock_name, rocks):
        rocks.append(rock_name)
        parentList.append(parent)
    return [rocks, rock_name]

In [4]:
list3rd = bsoup3.find(id="toc").find_next_siblings("h2", limit=3)
rocks = []
parent = []
mainParent = []
df1 = pd.DataFrame()
for item in list3rd:
    rockCategory = item.find("span").text
    rockCategory = str(rockCategory) + " rocks"
    for i in item.find_next_sibling("dl").find_all("dd"):
        mainParent.append(rockCategory)
        #for subchildelements
        if i.findAll("dl"):
            rocks, subparent = get_rock_name(i, rocks, parent, rockCategory)
            for subelements in i.find("dl").find_all("dd"):
                rocks,roc = get_rock_name(subelements, rocks, parent, subparent)
        #without having any child elements    
        else:
            rocks = get_rock_name(i, rocks, parent, rockCategory)[0]
    df1 = pd.DataFrame({"Rocks":rocks, "Parent": parent, "MainParent":mainParent, "Relation": "subClass"})
print(df1)

              Rocks             Parent         MainParent  Relation
0          Andesite      Igneous rocks      Igneous rocks  subClass
1       Anorthosite      Igneous rocks      Igneous rocks  subClass
2            Aplite      Igneous rocks      Igneous rocks  subClass
3            Basalt      Igneous rocks      Igneous rocks  subClass
4           Adakite             Basalt      Igneous rocks  subClass
..              ...                ...                ...       ...
124           Slate  Metamorphic rocks  Metamorphic rocks  subClass
125         Suevite  Metamorphic rocks  Metamorphic rocks  subClass
126  Talc carbonate  Metamorphic rocks  Metamorphic rocks  subClass
127       Soapstone     Talc carbonate  Metamorphic rocks  subClass
128     Whiteschist  Metamorphic rocks  Metamorphic rocks  subClass

[129 rows x 4 columns]


# 2nd link

In [5]:
#2nd link, 'https://www.sandatlas.org/rock-types/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

content = requests.get('https://www.sandatlas.org/rock-types/', headers=headers).text

In [6]:
bsoup = BeautifulSoup(content, 'html.parser')

In [7]:
list1234 = bsoup.find("a",attrs={"name": "Contents"}).find_all_next("p", limit=3)
df2 = pd.DataFrame()
rocks = []
parent = []
mainParent = []
for index, rClass in enumerate(list1234):
    parentClass = rClass.find("strong").text
    if (index != 0):
        rockSubClass = rClass.find_all("a")[1:]
    else:
        rockSubClass = rClass.find_all("a")
    for i in rockSubClass:
        rocks.append(i.text)
        parent.append(parentClass)
        mainParent.append(parentClass)
    df2 = pd.DataFrame({"Rocks":rocks, "Parent": parent, "MainParent":mainParent, "Relation": "subClass"})
print(df2)

           Rocks             Parent         MainParent  Relation
0       Andesite      Igneous rocks      Igneous rocks  subClass
1    Anorthosite      Igneous rocks      Igneous rocks  subClass
2         Aplite      Igneous rocks      Igneous rocks  subClass
3         Basalt      Igneous rocks      Igneous rocks  subClass
4       Basanite      Igneous rocks      Igneous rocks  subClass
..           ...                ...                ...       ...
96      Shungite  Metamorphic rocks  Metamorphic rocks  subClass
97         Skarn  Metamorphic rocks  Metamorphic rocks  subClass
98         Slate  Metamorphic rocks  Metamorphic rocks  subClass
99     Soapstone  Metamorphic rocks  Metamorphic rocks  subClass
100      Suevite  Metamorphic rocks  Metamorphic rocks  subClass

[101 rows x 4 columns]


# 3rd link

In [8]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

html_content = requests.get('https://en.wikipedia.org/wiki/List_of_rock_types', headers=headers).text
soup = BeautifulSoup(html_content, 'html.parser')

In [9]:
list_items = soup.find(id="toc").find_all_next("h2", limit=4)

In [10]:
df3 = pd.DataFrame()
rocks = []
parent = []
mainParent = []
for index,item in enumerate(list_items[1:]):
    rockCat = item.find("span").text
    for rock in item.find_next("ul").find_all("li"):
        mainParent.append(rockCat)
        if(rock.findAll("ul")):
            if rock.find("a").text in rocks:
                continue
            else:
                subparent = rock.find("a").text
                rocks.append(subparent)
                parent.append(rockCat)
            for subelements in rock.find("ul").findAll("li"):
                if subelements.find("a").text in rocks:
                    continue
                else:
                    rocks.append(subelements.find("a").text)
                    parent.append(subparent)
            
        else:
            if rock.find("a").text in rocks:
                continue
            else:
                rocks.append(rock.find("a").text)
                parent.append(rockCat)
    df3 = pd.DataFrame({"Rocks":rocks, "Parent": parent, "MainParent":mainParent, "Relation": "subClass"})
print(df3)

                       Rocks             Parent         MainParent  Relation
0                    Adakite      Igneous rocks      Igneous rocks  subClass
1                   Andesite      Igneous rocks      Igneous rocks  subClass
2    Alkali feldspar granite      Igneous rocks      Igneous rocks  subClass
3                Anorthosite      Igneous rocks      Igneous rocks  subClass
4                     Aplite      Igneous rocks      Igneous rocks  subClass
..                       ...                ...                ...       ...
140                  Suevite  Metamorphic rocks  Metamorphic rocks  subClass
141           Talc carbonate  Metamorphic rocks  Metamorphic rocks  subClass
142                Soapstone     Talc carbonate  Metamorphic rocks  subClass
143                Tectonite  Metamorphic rocks  Metamorphic rocks  subClass
144              Whiteschist  Metamorphic rocks  Metamorphic rocks  subClass

[145 rows x 4 columns]


# Data Aggregation

In [11]:
frames = [df1, df2, df3]

In [12]:
result = pd.concat(frames)

In [13]:
print(result) 

              Rocks             Parent         MainParent  Relation
0          Andesite      Igneous rocks      Igneous rocks  subClass
1       Anorthosite      Igneous rocks      Igneous rocks  subClass
2            Aplite      Igneous rocks      Igneous rocks  subClass
3            Basalt      Igneous rocks      Igneous rocks  subClass
4           Adakite             Basalt      Igneous rocks  subClass
..              ...                ...                ...       ...
140         Suevite  Metamorphic rocks  Metamorphic rocks  subClass
141  Talc carbonate  Metamorphic rocks  Metamorphic rocks  subClass
142       Soapstone     Talc carbonate  Metamorphic rocks  subClass
143       Tectonite  Metamorphic rocks  Metamorphic rocks  subClass
144     Whiteschist  Metamorphic rocks  Metamorphic rocks  subClass

[375 rows x 4 columns]


In [14]:
result1 = result.drop_duplicates(['Rocks'])
result1 = result1.sort_values(by=['Parent'])

In [15]:
result1.to_csv("outputrockremovingDuplicates.csv", index=False) 

# Save to existing ontology

In [29]:
import pandas as pd
from owlready2 import *
onto = get_ontology("file://C:/Users/Lenovo/Desktop/UNI-Koblenz-Landua/Research/Coding/GeologicalTaxonomy.owl").load()

In [30]:
with onto:
    class Igneous_rocks(onto.Rock):
        pass
    class Sedimentary_rocks(onto.Rock):
        pass
    class Metamorphic_rocks(onto.Rock):
        pass

In [31]:
data = pd.read_csv("outputrockremovingDuplicates.csv")

In [32]:
for index, row in data.iterrows():
    with onto:
        rock = row['Rocks'].replace(" ","_")
        subParent = row['Parent'].replace(" ","_")
        superParent = row['MainParent'].replace(" ","_")
        for rockCategory in list(onto.classes()):
            if (superParent == rockCategory.name) and (subParent != superParent):
                NewClass = type(subParent,(rockCategory,),{})
        for rockCat in list(onto.classes()):
            if(rockCat.name==subParent):
                parent_class = rockCat
                break 
        NewClass = type(rock,(rockCat,),{})
onto.save(file = "./geological-taxonomy.owl", format = "rdfxml")